In [1]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Functional Programming


## Specifying algorithms

We need a way to specify both **what** an algorithm does and **how** it does it.

**Algorithm specification**: defines what an algorithm should do.

> Given a sequence $A$ of $n$ elements, return a sequence $B$ such that $B[i] \leq B[j]$ for all $0 \leq i \le j \le n$

May also include **cost specification**, e.g. $O(n \log n)$ work and. $O(\log^2 n)$ span.

**Algorithm implementation** (or just **algorithm**): defines **how** an algorithm works.

This could be real, working code, or pseudo-code.



## Functional languages

In functional languages, functions act like mathematical functions.

Two key properties:

1. function maps an input to an output $f : X \mapsto Y$
    - no **side effects**
    
    
2. function can be treated as values
    - function A can be passed to function B

## Pure function

A function is **pure** if it maps an input to an output with no **side effects.**

A computation is **pure** if all of its functions are pure.


In [2]:
def double(value):
    return 2 * value

double(10)

20

We can view the `double` function as a mathematical function, defined by the mapping:

$$ \{(0, 0), (1, 2), (2, 4), \ldots \}$$

versus...

In [3]:
def append_sum(mylist):
    return mylist.append(sum(mylist))

mylist = [1,2,3]
append_sum(mylist)
mylist

[1, 2, 3, 6]

This has the side effecet of changing (or *mutating*) `mylist`.

though compare with...

In [4]:
def append_sum(mylist):
    return list(mylist).append(sum(mylist))

mylist = [1,2,3]
append_sum(mylist)
mylist

[1, 2, 3]

Almost all "real" computations have some side effects. Consider:

In [5]:
def do_sum(mylist):
    total = 0
    for v in mylist:
        total += v
    return total

`do_sum` has the side effect of modifying `total`. But, this effect is not visible outside of `do_sum`, due to variable scoping.

> **benign effect:** a side-effect that is not observable from outside of the function.

A function with benign effects is still considered pure.

## Why is pure computation good for parallel programming?

  Recall our **race condition** example:

In [6]:
from multiprocessing.pool import ThreadPool

def in_parallel(f1, arg1, f2, arg2):
    with ThreadPool(2) as pool:
        result1 = pool.apply_async(f1, [arg1])  # launch f1
        result2 = pool.apply_async(f2, [arg2])  # launch f2
        return (result1.get(), result2.get())   # wait for both to finish
    
total = 0

def count(size):
    global total
    for _ in range(size):
        total += 1
    
def race_condition_example():
    global total
    in_parallel(count, 100000,
                count, 100000)
    print(total)
    
race_condition_example()

140632


The `count` function has a side-effect of changing the global variable `total`.

## Heisenbugs

<img src="figures/heisenberg.jpg" width="10%"/>

- Race conditions can lead to bugs that only appear, e.g., 1 out of 1000 runs of the program.
- Reference to Heisenberg uncertainty principal (the bug disappears when you study it, but reappears when you stop studying it)


More generally, if we want to parallelize two functions $f(a)$ and $g(b)$, we want the same result **no matter which order they are run in.**

> Because of the lack of side-effects, pure functions satisfy this condition.

## Data Persistence

In pure computation no data can ever be overwritten, only new data can be created.   

Data is therefore always **persistent**
  —if you keep a reference to a data structure, it will always be there and in the same state as it started.

### Isn't this horribly space inefficient?

> garbage collection

## Functional languages

In functional languages, functions act like mathematical functions.

Two key properties:

1. function maps an input to an output $f : X \mapsto Y$
    - no **side effects**
    
    
2. function can be treated as values
    - function A can be passed to function B

## Functions as values

Many languages allow functions to be passed to other functions.

Functions as "first-class values."

In [7]:
def double(value):
    return 2 * value

def double_and_sum(double_fn, vals):
    total = 0
    for v in vals:
        total += double_fn(v)
    return total

# pass the function double to the function double_and_sum
double_and_sum(double, [1,2,3]) 
# 1*2 + 2*2 + 3*3

12

`double_and_sum` is called a **higher-order function**, since it takes another function as input.

Why is this useful?

In [8]:
def map_function(function, values):
    for v in values:
        yield function(v)

list(map_function(double, [1,2,3]))

[2, 4, 6]

In [9]:
def square(value):
    return value * value

list(map_function(square, [1,2,3]))

[1, 4, 9]

In [10]:
list(map_function(double, map_function(square, [1,2,3])))

[2, 8, 18]

- If we know that `function` is pure, then we can trivially parallelize `map_function` for many inputs.



- By using higher-order functions, we can define a few primitive, high-order functions that will make it easier to reason about and analyze run-time of parallel computations.

## SPARC

Our textbook uses a pseudo code language called **SPARC**
- based on [Standard ML](https://www.cs.cmu.edu/~rwh/isml/book.pdf)
- functional language

When possible, we will also show Python versions of key algorithms.


## Example SPARC program


<br><br>
<p> <span>\[\begin{array}{l}  
\texttt{let}\\   
~~~~x = 2 + 3\\  
~~~~f (w) = (w * 4, w - 2)\\  
~~~~(y,z) = f(x-1)\\  
\texttt{in}\\   
~~~~x + y + z\\  
\texttt{end}   
\end{array}\]</span></p>
<br><br>

<br><br>
**binding**: associate entities (data or code) with identifiers.

<br>

**let expression:**

**let**  
$\:\: b^+$  
**in**  
$\:\:e$  
**end**

Expression $e$ is applied using the bindings defined inside **let**.

<br><br>
**expression** *e*: describes a computation  
- **evaluating** an expression produces its value

<br><br>
$x = 2 + 3 = 5$  
$f(4) \rightarrow (16, 2)$  
$x + y + z= 5 + 16 + 2 = 23$







### What does this do?

<p><span class="math display">\[\begin{array}{l}  
\texttt{let}\\  
~~~~f(i) = \texttt{if} ~(i < 2) ~\texttt{then}~ i ~\texttt{else}~ i  *   
f(i - 1) \\  
\texttt{in} \\   
~~~~f(5) \\  
\texttt{end}   
\end{array}\]</span> </p>



In [4]:
def factorial(i):
    if i<2:
        return i
    else:
        return i*factorial(i-1)
    

factorial(8)

40320

## Lambda Calculus 

Consists of expressions $e$ in one of three forms:

1. a **variable**, e.g., $x$
2. a **lambda abstraction**, e.g., $(\lambda \: x \: . \: e)$, where $e$ is a function body.
3. an **application**, written $(e_1, e_2)$ for expressions $e_1$, $e_2$.

```python
factorial = lambda i: i if i < 2 else i*factorial(i-1)
```

In [6]:
# lambda functions exist in Python.
# these are anonymous functions (no names)
# Here, e_2 is a variable.
(lambda x: x*x)(12)

144


$
(\lambda \: x \: . \: x * x) \:  10 \mapsto 10 * 10 \mapsto 100
$

We can also chain functions together. E.g., $e_2$ can be another function.

In [1]:
def compose(g, f):
    """
    Returns a **function** that composes f and g
    """
    return lambda x: g(f(x))  # different from just: g(f(x))

def meter2cm(d):
    return d * 100

def cm2inch(d):
    return d / 2.54


# how many inches in a meter?
meter2inch = compose(meter2cm, cm2inch)
meter2inch(1)

39.370078740157474

## Composition

**sequential composition**: $(e_1, e_2)$

**parallel composition**: $(e_1 \: || \: e_2)$

e.g.

$\mathtt{lambda} \: (x,y) . (x * x, y * y)$  

vs

$\mathtt{lambda} \: (x,y) . (x * x \: || \: y * y)$

### Currying

> Convert a function of $n$ variables into a sequence of functions with 1 argument each.


**Why?**

<br><br>
- Get specialized functions from more general functions by using composition.
- DRY: no need to repeat function arguments
- Lambda calculus: can define a programming language that only allows functions of one argument
  - easier for proofs!

In [ ]:
def curry(f):
    """
    Given a function f of two variables,
    return a function g that binds the first variable
    and returns a function of the second variable.
    """
    def g(x):      # nested function 1
        def h(y):  # nested function 2
            return f(x, y) 
        return h # bind x and return function of y
    return g

def f(x,y):
     return x + y**2

print(f)
print(curry(f))         # returns f'n g. input: x, output function of y         
print(curry(f)(10))     # returns f'n h. input: y, output f(10,y)
print(curry(f)(10)(20)) # returns f(10,20)
print(curry(f)(10)(3))  # returns f(10,3)